Below: the stuff from huggingface deep rl lesson 7

In [ ]:
import gym
import pybullet_envs

import os

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

import torch 
from torch import nn

In [ ]:
env_id = "AntBulletEnv-v0"
# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space

In [ ]:
env = make_vec_env(env_id, n_envs=4)

# Adding this wrapper to normalize the observation and the reward
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.)

In [ ]:
model = A2C(policy = "MlpPolicy",
            env = env,
            gae_lambda = 0.9,
            gamma = 0.99,
            learning_rate = 0.00096,
            max_grad_norm = 0.5,
            n_steps = 8,
            vf_coef = 0.4,
            ent_coef = 0.0,
            tensorboard_log = "./tensorboard",
            policy_kwargs=dict(
            log_std_init=-2, ortho_init=False),
            normalize_advantage=False,
            use_rms_prop= True,
            use_sde= True,
            verbose=1)

In [ ]:
#Takes tens of minutes
model.learn(2_000_000)
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-AntBulletEnv-v0")
env.save("vec_normalize.pkl")

End Huggingface stuff.

Now let's do it with cartpole to see how quickly it grasps a simple env.

Was going to adapt the huggingface stuff but this is from the stable baselines website verbatim

In [ ]:
import gym

#from stable_baselines3.common.policies import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import A2C

# Parallel environments
env = make_vec_env('CartPole-v1', n_envs=4)

model = A2C("MlpPolicy", env, verbose=1)
for i in range(1000):
    model.learn(total_timesteps=1000)

    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    if mean_reward > 110:
        print(f"took {i * 1000} timesteps to reach mean reward 100")
        break;
#model.learn(total_timesteps=100000)
model.save("a2c_cartpole")

del model # remove to demonstrate saving and loading

model = A2C.load("a2c_cartpole")

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

OK now let's try Pong

In [ ]:
import gym

#from stable_baselines3.common.policies import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import A2C

# Parallel environments
env = make_vec_env('ALE/Pong-v5', n_envs=4)

model = A2C("CnnPolicy", env, verbose=1)
#model = A2C.load("a2c_pong")

from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%Y%m%d%H%M")
save_file = f"a2c_pong-{dt_string}"
print(f"Save file: {save_file}")

reward_milestones = []
reward_target = -20

In [ ]:
n = 100_000
for i in range(1000):
    model.learn(total_timesteps=n, reset_num_timesteps=False)

    mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
    print(f"Now at {n * (i+1)} timesteps")
    if mean_reward > reward_target:
        print(f"took {n * (i+1)} timesteps to reach mean reward {reward_target}")
        reward_milestones.append({"reward": reward_target, "timesteps": (n*(i+1))})
        reward_target+=1
        break;

#model.learn(total_timesteps=1_000_000)

model.save(save_file)


In [ ]:
print(reward_milestones)
print(f"Next reward target: {reward_target}")

In [ ]:
#del model # remove to demonstrate saving and loading

#model = A2C.load("a2c_pong")

obs = env.reset()
for i in range(100000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
env.reset()